In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

import sys
sys.path.append('../')

from wavy import panel
from wavy import side
from wavy import block
from wavy import nn
from wavy import plot

In [2]:
df = pd.read_pickle('processed.pkl')

In [3]:
df.head(5)

LNC                                                    MAS  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2005-12-21  38.127885  39.324626  38.120589  39.207870  2190100.0  18.991695   
2005-12-22  39.185984  39.324630  38.879500  39.040039  1095500.0  19.085703   
2005-12-23  39.061931  39.397604  38.959771  39.324631   588800.0  18.954090   
2005-12-27  39.514374  39.514374  38.981676  39.237080   932700.0  18.929008   
2005-12-28  39.455973  39.696782  39.244353  39.310028   427200.0  18.853798   

                                                        ...       CSX  \
                 High        Low      Close     Volume  ...      Open   
Date                                                    ...             
2005-12-21  19.154659  18.903944  19.085712  1384200.0  ...  6.245725   
2005-12-22  19.298810  18.910202  18.929007  1576900.0  ...  6.381336   
2005-12-23  18.954090  18.684571  18.816196  2090500.0  ...  6.297691   
2005-12-27  18.991687  18.803650  18.847525  1609000.0  ...  6.520750   
2005-12-28  18.935281  18.747245  18.803656  2026300.0  ...  6.419358   

                                                            CI             \
                High       Low     Close     Volume       Open       High   
Date                                                                        
2005-12-21  6.399080  6.245725  6.362326  8358000.0  36.945102  37.462575   
2005-12-22  6.420626  6.361058  6.410487  5344800.0  37.409841  37.630677   
2005-12-23  6.520752  6.297691  6.468790  3630000.0  37.376876  37.432909   
2005-12-27  6.527087  6.369931  6.400348  4905000.0  37.195584  37.455973   
2005-12-28  6.459914  6.363592  6.419358  3578400.0  37.241735  37.449383   

                                             
                  Low      Close     Volume  
Date                                         
2005-12-21  36.895658  37.370289  2805900.0  
2005-12-22  37.073647  37.327442  2296800.0  
2005-12-23  36.905546  37.261517  1686000.0  
2005-12-27  36.961570  37.116482  1917900.0  
2005-12-28  37.109892  37.251621  1248900.0  

[5 rows x 2085 columns]

In [14]:
panel_ = panel.from_data(df, lookback = 5, horizon = 3, gap = 2, assets = ['AAPL', 'MSFT'], channels= ['Close', 'Open'])

In [18]:
panel_.x[0]

AAPL                 MSFT           
               Close      Open      Close       Open
Date                                                
2005-12-21  2.246069  2.218566  19.475122  19.577126
2005-12-22  2.261960  2.258598  19.373114  19.460543
2005-12-23  2.241485  2.266543  19.409552  19.322122
2005-12-27  2.268378  2.261349  19.278402  19.438692
2005-12-28  2.248209  2.275712  19.227409  19.314840

In [16]:
panel_.y[0]

AAPL                 MSFT           
               Close      Open      Close       Open
Date                                                
2006-01-03  2.284267  2.211843  19.555256  19.125390
2006-01-04  2.290990  2.295879  19.649986  19.504270
2006-01-05  2.272961  2.286712  19.664555  19.642696

In [24]:
panel_.plot_block(0)

In [25]:
panel_.plot_slider(10)

In [26]:
panel_.x.plot_slider()

In [27]:
panel_.x[0].plot()

In [ ]:
panel_.plot_slider()

In [33]:
panel_.y[0]

AAPL                 MSFT           
               Close      Open      Close       Open
Date                                                
2006-01-03  2.284267  2.211843  19.555256  19.125390
2006-01-04  2.290990  2.295879  19.649986  19.504270
2006-01-05  2.272961  2.286712  19.664555  19.642696

In [29]:
panel_.y[1]

AAPL                 MSFT           
               Close      Open      Close       Open
Date                                                
2006-01-04  2.290990  2.295879  19.649986  19.504270
2006-01-05  2.272961  2.286712  19.664555  19.642696
2006-01-06  2.331634  2.299547  19.606268  19.591696

In [31]:
a = panel_.y.side_shift(1)

In [32]:
a[0]

AAPL       MSFT     
           Close Open Close Open
Date                            
2006-01-03   NaN  NaN   NaN  NaN
2006-01-04   NaN  NaN   NaN  NaN
2006-01-05   NaN  NaN   NaN  NaN

In [ ]:
panel_.x[0].shift(1)

In [ ]:
side_diff = panel_.y.side_diff(1)

In [ ]:
side_diff[2]

In [ ]:
side_shift = panel_.y.side_shift(1)
side_diff = panel_.y.side_diff(1)

In [ ]:
side_shift[1]

In [ ]:
side_diff[1]

In [ ]:
pct_change = side_diff / panel_.y

In [ ]:
pct_change[1]

In [ ]:
b = panel_.y.side_pct_change(1)

In [ ]:
b[1]

In [ ]:
a = pd.DataFrame(data=[1,2,3,4,5], index=[0,1,2,3,4])

In [ ]:
a

In [ ]:
b = a+2
b

In [ ]:
b/a

In [ ]:
pct = panel_.y.side_pct_change(1)

In [ ]:
panel_.x[1]

In [ ]:
panel_.y[1]

In [ ]:
a1 = panel_.y
a0 = panel_.y.side_shift(1)

In [ ]:
a1[1]

In [ ]:
a0[1]

In [ ]:
a = panel_.y.side_diff(1)

In [ ]:
a[1]

In [ ]:
a1[1]-a0[1]

In [ ]:
(a1[1]-a0[1])/a0[1]

In [ ]:
panel_.y = panel_.y.side_diff(1)

In [ ]:
panel_.x[1871]

In [ ]:
np.linspace(0,len(panel_.x.blocks),100, dtype=int)

In [ ]:
import plotly.graph_objects as go
import numpy as np
import plotly.express as px


import pandas as pd
import plotly as px
pd.set_option("multi_sparse", True)  # To see multilevel indexes
pd.options.plotting.backend = "plotly"
from plotly.subplots import make_subplots

cmap = px.colors.qualitative.Plotly

# Create figure
# fig = go.Figure()
fig = make_subplots(rows=len(panel_.channels), cols=len(panel_.assets), subplot_titles=panel_.assets, shared_xaxes=True)

graph_number = len(panel_.channels) * len(panel_.assets)

# Add traces, one for each slider step
len_ = np.linspace(0,len(panel_.x.blocks),100, dtype=int, endpoint=False)
for step in len_: #np.arange(len(panel_.x.blocks)):

    for j, channel in enumerate(panel_.channels):
        c = cmap[j]
        for i, asset in enumerate(panel_.assets):

            showlegend = i <= 0

            x_df = panel_.x.blocks[step].filter(assets=asset, channels=channel)
            index = x_df.index
            values = x_df.values.flatten()

            x_trace = go.Scatter(visible=False,
                                x=index,
                                y=values,
                                line=dict(width=2, color=c), showlegend=showlegend, name=channel)

            # x_trace = go.Scatter(x=index, y=values,
            #                     line=dict(width=2, color=c), showlegend=showlegend, name=channel)

            fig.add_trace(x_trace, row=j+1, col=i+1)

            dt_all = pd.date_range(start=index[0],end=index[-1])
            dt_obs = [d.strftime("%Y-%m-%d") for d in index]
            dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]
            fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

# Make 10th trace visible
for i in range(graph_number):
    fig.data[i].visible = True

# Create and add slider
steps = []
for i in range(len(len_)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
            {"title": "Block " + str(len_[i])}],  # layout attribute
    )

    for g in range(graph_number):
        step["args"][0]["visible"][i*graph_number+g] = True  # Toggle i'th trace to "visible"

    steps.append(step)


sliders = [dict(
    active=0,
    # currentvalue={"prefix": "Block: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()

In [ ]:
panel_.x -= panel_.x.side_shift(1)
panel_.y -= panel_.y.side_shift(1)

In [ ]:
panel_.y.shape

In [ ]:
panel_ = panel_.dropna()

In [ ]:
panel_.y.shape

In [ ]:
model = nn.LinearRegression(panel_)

In [ ]:
model.fit()

In [ ]:
predicted = model.predict()

In [ ]:
predicted[0]

In [ ]:
model.evaluate()

In [ ]:
len(a)

In [ ]:
panel_.x[0]

In [ ]:
all(a[0].index)

In [ ]:
all(a[1].index.isna())

In [ ]:
type(a[0])

In [ ]:
print(a[0])

In [ ]:
a[1].is_valid

In [ ]:
a[1]+a[2]

In [ ]:
type(a[5].as_dataframe())

In [ ]:
a5 = a[5].as_dataframe()
a1 = a[1].as_dataframe()

a5.index = a5.index.astype(str)
a1.index = a1.index.astype(str)


In [ ]:
pd.DataFrame(data=a1.values+a5.values, columns=a1.columns, index=a1.index)

In [ ]:
a1.add(a5)

In [ ]:
len(panel_.x)

In [ ]:
[None] * 2 + [1,2,3] + [None]*2

In [ ]:
panel_

In [ ]:
panel_.x[0]

In [ ]:
panel_.y[0]

In [ ]:
panel_.y.first

In [ ]:
a = panel_.y + panel_.y

In [ ]:
a.first

In [ ]:
c = panel_.y.__add__(panel_.y)

In [ ]:
c.first

In [ ]:
panel_.x.first

In [ ]:
panel_.x[1]

In [ ]:
panel_.x.shift()[0]

In [ ]:
a = pd.DataFrame([[1, 2], [3,4]])

In [ ]:
a.__add__(a)

# Graph types

Separate X from Y

In [ ]:
panel_.plot(2)

In [ ]:
panel_.plot(3)

In [ ]:
panel_.x.first

In [ ]:
panel_.x[0]

In [ ]:
panel.x[0].index

In [ ]:
panel_.x.plot(2)

In [ ]:
panel.x[0]

In [ ]:
panel_.x[2].plot()

In [ ]:
panel.y[0]

In [ ]:
panel.set_training_split()

In [ ]:
panel.x.first

In [ ]:
panel.shape

# Baseline

In [ ]:
baseline = nn.Baseline(panel, metrics=['SE', 'MSE', 'RMSE'])

In [ ]:
predicted = baseline.predict()

In [ ]:
panel.test.y[0]

In [ ]:
predicted[2]

In [ ]:
predicted_val = baseline.predict_val()

In [ ]:
panel.val.y[0]

In [ ]:
predicted_val[2]

In [ ]:
baseline.evaluate('test')

In [ ]:
# m = tf.keras.metrics.MeanSquaredError()
# m.update_state(panel.test.y.tensor3d, predicted.tensor3d)
# m.result().numpy()

# DenseModel

In [ ]:
densemodel = nn.DenseModel(panel, model_type="regression")

In [ ]:
densemodel.fit()

In [ ]:
densemodel.model.summary()

In [ ]:
predicted = densemodel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
densemodel.evaluate('test')

In [ ]:
import numpy as np
import pandas as pd
import plotly as px
import plotly.graph_objects as go
import plotly.express as px

pd.set_option("multi_sparse", True)  # To see multilevel indexes
pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots

def pair_plot(panel, idx):

    cmap = px.colors.qualitative.Plotly

    fig = make_subplots(rows=len(panel.channels), cols=len(panel.assets), subplot_titles=panel.assets, shared_xaxes=True)

    for j, channel in enumerate(panel.channels):
        c = cmap[j]
        for i, asset in enumerate(panel.assets):

            showlegend = i <= 0
            x_df = panel.x[idx].filter(assets=asset, channels=channel)
            y_df = panel.y[idx].filter(assets=asset, channels=channel)
            # y_df = panel.y[idx].filter(assets=asset, channels=channel)

            x_trace = go.Scatter(x=x_df.index, y=x_df.values.flatten(),
                            line=dict(width=2, color=c), showlegend=showlegend, name=channel)

            y_trace = go.Scatter(x=y_df.index, y=y_df.values.flatten(),
                                line=dict(width=2, dash='dot', color=c), showlegend=False)

            fig.add_trace(x_trace, row=j+1, col=i+1)
            fig.add_trace(y_trace, row=j+1, col=i+1)

    fig.update_layout(showlegend=True)
    fig.show()

In [ ]:
panel.x.as_dataframe()

In [ ]:
# panel = panel.filter('AAPL', ['High', 'Low'])
pair_plot(panel, 0)

In [ ]:
panel = panel.filter('LNC', ['Open', 'High'])
panel[0].plot()
panel.y_ = model.predict()

# ConvModel

In [ ]:
convmodel = nn.ConvModel(panel, model_type="regression")

In [ ]:
convmodel.fit()

In [ ]:
convmodel.model.summary()

In [ ]:
predicted = convmodel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
convmodel.evaluate()

# LinearRegression

In [ ]:
linearRegression = nn.LinearRegression(panel)

In [ ]:
linearRegression.fit()

In [ ]:
predicted = linearRegression.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
linearRegression.evaluate()

# Teste SeparateAssetModel

In [ ]:
panel.val.x[0]

In [ ]:
test_splits = panel.val.x._split_assets()

In [ ]:
x_train = [x.tensor3d for x in test_splits]

In [ ]:
# pd.DataFrame(x_train)

In [ ]:
test_splits[0].tensor3d[0]

In [ ]:
test_splits[0].tensor4d[0]

In [ ]:
pd.DataFrame(panel.val.x.tensor4d)

# SeparateAssetModel

In [ ]:
separateAssetModel = nn.SeparateAssetModel(panel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

In [ ]:
separateAssetModel.fit()

In [ ]:
separateAssetModel.model.summary()

In [ ]:
predicted = separateAssetModel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
separateAssetModel.evaluate()

# SeparateAssetConvModel

In [ ]:
separateAssetConvModel = nn.SeparateAssetConvModel(panel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

In [ ]:
separateAssetConvModel.fit()

In [ ]:
separateAssetConvModel.model.summary()

In [ ]:
predicted = separateAssetConvModel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
separateAssetConvModel.evaluate()

# Test Docstrings

In [ ]:
densemodel = nn.DenseModel()